In [1]:
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
import torch


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [38]:
!pip install streamlit

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 56.3 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 191.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.9/188.9 kB 54.6 MB/s eta 0:00:00
  Attempting uninstall: typin

In [39]:

import streamlit as st

In [1]:
!pip install transformers datasets faiss-gpu torch sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 15.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you wan

In [2]:
!pip install accelerate==1.0


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [40]:
# Step 1: Load and Preprocess Dataset (split by lines instead of paragraphs)
@st.cache_resource
def load_and_tokenize(file_path):
    with open(file_path, 'r') as file:
        text = file.read()  # Read the entire content of the file
    # Split the text into lines
    lines = text.split('. ')  # Assuming period as line separator, adjust as needed
    return lines

In [4]:
file_path = 'hv_english3.txt'
preprocessed_lines = load_and_tokenize(file_path)

In [42]:
@st.cache_resource
def load_retriever_model():
    return SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [43]:
retriever_model = load_retriever_model()

2024-10-17 19:35:26.951 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 19:35:27.032 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]
2024-10-17 19:35:27.033 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 19:35:27.034 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 19:35:27.536 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 19:35:27.537 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 19:35:28.425 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 19:35:28.426 Thread 'MainThread': missing S

In [6]:
# Generate embeddings for each line in your dataset
line_embeddings = retriever_model.encode(preprocessed_lines, convert_to_tensor=True)

In [7]:
# Step 3: Build FAISS index
index = faiss.IndexFlatL2(line_embeddings.shape[1])
faiss.normalize_L2(line_embeddings.cpu().detach().numpy())  # Normalize embeddings for better similarity search
index.add(line_embeddings.cpu().detach().numpy())

In [8]:
# Step 4: Search function to retrieve the top-1 most relevant line
def retrieve_relevant_passages_with_context(query, k=3, context_size=15):
    # Generate embedding for the query
    query_embedding = retriever_model.encode([query], convert_to_tensor=True)
    faiss.normalize_L2(query_embedding.cpu().detach().numpy())  # Normalize query embedding
    
    # Search the FAISS index for top-k nearest neighbors
    _, relevant_indices = index.search(query_embedding.cpu().detach().numpy(), k)
    
    # Retrieve the relevant lines and surrounding context
    relevant_index = relevant_indices[0][0]  # Get the index of the most relevant line
    start_index = max(relevant_index - context_size, 0)  # Ensure we don't go below 0
    end_index = min(relevant_index + context_size + 1, len(preprocessed_lines))  # Ensure we don't exceed dataset length
    
    # Extract 10 lines before and after the relevant paragraph
    context_lines = preprocessed_lines[start_index:end_index]
    return " ".join(context_lines)  # Join lines back into a single string for context

In [47]:
# Step 5: Load LLaMA model for information extraction
#model_id = "AI-Sweden-Models/Llama-3-8B-instruct"

In [9]:
access_token = "XXXXXXX"


In [10]:
model_id ='meta-llama/Llama-3.1-8B-Instruct'


In [ ]:
llama_pipeline = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    #use_auth_token=access_token  # Only here!
)


In [32]:
def correct_text_with_llama(text, query, max_new_tokens):
    # Combine text and query into messages suitable for the LLaMA model
    messages = [
        {"role": "system", "content": "You are an expert chatbot assistant who answers specific questions analysing only the text provided to you.Please provide a concise answer without any formalities like 'Best regards' or repeated suggestions. "}, 
        {"role": "user", "content": f"Answer this question: {query}, from this text: \"{text}\""}
    ]

    # Define terminators for generation stopping
   #terminators = [
   #       llama_pipeline.tokenizer.eos_token_id,
   #       llama_pipeline.tokenizer.convert_tokens_to_ids("")
   #]
    eos_token_id = llama_pipeline.tokenizer.eos_token_id
    terminators = eos_token_id
    repetition_penalty=1.2
    # Generate output from the LLaMA model
    output = llama_pipeline(
        messages,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.8,
        repetition_penalty=repetition_penalty
    )

    # Extract the assistant's reply from the output
    return output[0]['generated_text']



In [33]:
# Step 6: Generate Response by First Retrieving Relevant Context, Then Using LLaMA for Extraction
def generate_response(query, k, context_size):
    # Step 1: Retrieve the most relevant passages with 10 lines before and after
    relevant_context = retrieve_relevant_passages_with_context(query, k, context_size)
    print(relevant_context)
    # Step 2: Pass the context to LLaMA to extract the specific relevant information
    extracted_information = correct_text_with_llama(relevant_context, query, max_new_tokens=4096)
    
    # Return the extracted information as the final response
    return extracted_information


In [34]:
 #Step 7: Test with a Query
query = "do university west offer any internship program?"
response = generate_response(query, k=5, context_size=10)  # Retrieve top-1 most relevant paragraph + 10 lines before/after
print("Chatbot Response:", response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


the eu card entitles you to medically necessary healthcare benefits in euees countries and switzerland without any additional cost apart from a possible patient fee the eu card grants you access to hospital care medical treatment dental care medications etc provided within the public healthcare system in the country you are in the card must always be shown when seeking medical treatment.
 download the erasmus app before your departurethe european commission has developed an app for students traveling abroad within euees the aim is to create a platform where students can find information about the program and related tasks the app is managed by the european commission and you can find more information here.
 website tips
 erasmus intern  companies and organizations offer internship placements for erasmus internships.
 on the swedish council of higher educations website you can find internship opportunities within europe.
 espa uk arranges internships in the united kingdom for university

In [29]:
 #Step 7: Test with a Query
query = "how to apply for scholarship in university west?"
response = generate_response(query, k=5, context_size=10)  # Retrieve top-1 most relevant paragraph + 10 lines before/after
print("Chatbot Response:", response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


fördjupning lambda and integrationsvem ska delta tekniker
 satyapal mahade about  sicomap
 regulations for award of sparbanksstiftelsen väst scholarship 2024
 swedbank in collaboration with the swedish foundation sparbanksstiftelsen väst has donated 40 000 kr to university west for awarding students who have completed a study or internship period abroad within their education here the funds will be awarded as 2 scholarships of 20 000 sek each.
 university west will award these two scholarships to students who have completed a study or internship period abroad of more than 8 weeks.students in firstcycle studies bachelor level and students in secondcycle studies master level are welcome to apply.
 students who are currently studying at university west who have been abroad for study or internship during spring 2024 or are going for study or internship abroad during autum 2024 or spring 2025 and have not received any funds from university west can apply.
 the two students awarded the schol

In [28]:
 #Step 7: Test with a Query
query = "how much is the tuition fee for masters programme?"
response = generate_response(query, k=5, context_size=10)  # Retrieve top-1 most relevant paragraph + 10 lines before/after
print("Chatbot Response:", response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


this indicates a strong market for engineers within the field of ai automation and smart manufacturing as the development of ai solutions is key for implementing smart manufacturing.
 while large universities have several focus areas university wests small size and high quality allow us to focus on and dedicate resources to a few prioritisedselected areas such as production technology and processes this ensures a highquality programme as well as a warm and friendly atmosphere.
 your programme combines learning in production technology and computer science this allows you to build a unique expertise in both ai and automation entering the job market as an innovative engineer where you will shape new production systems and products.
 this combination is unavailable from most other universities.
 production systems
 automation and ai cloud automation and digital twins
 process control and optimal control in the manufacturing process
 sensor technology smart sensing and image analysis
 huma

In [35]:
#Step 7: Test with a Query
query = "please provide me university west official email/website?"
response = generate_response(query, k=5, context_size=10)  # Retrieve top-1 most relevant paragraph + 10 lines before/after
print("Chatbot Response:", response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


of course international cultural competency and english language ability are also important we have a diverse workforce and are building diverse teams.
 interview with mengting xie  alumna from china
 i currently work as an engineer and am the operational process development  system solution architect at national electric vehicle sweden nevs in trollhättan during a typical working day i get into the office early to go through my todo checklist in onenote in order to prioritize things i should do for the day during the early morning our team always has a short stand up meeting to communicate about the following questions
 what we did yesterday?
 what we will do today?
 what blocks us from maintaining our progress?
 as an agile organization we have adapted our way of working to share our progress help each other and react faster to changes.
 i work on multiple assignments during a day such as process development process communication business system development and maintenance itrelated 